In [5]:
from libpgm.nodedata import NodeData
from libpgm.graphskeleton import GraphSkeleton
from libpgm.discretebayesiannetwork import DiscreteBayesianNetwork
from libpgm.lgbayesiannetwork import LGBayesianNetwork
from libpgm.hybayesiannetwork import HyBayesianNetwork
from libpgm.dyndiscbayesiannetwork import DynDiscBayesianNetwork
from libpgm.tablecpdfactorization import TableCPDFactorization
from libpgm.sampleaggregator import SampleAggregator
from libpgm.pgmlearner import PGMLearner

import numpy as np
#create table
from prettytable import PrettyTable

#read csv
import pandas as pd

In [23]:
data=pd.read_csv("./Training/first_step_prepro/training_initial_sentimental.csv",sep='[;,|]', engine='python')
data.query('overall > 0',inplace=True)
data.query('clean > 0',inplace=True)
data.query('value >0',inplace=True)
data.query('room >0',inplace=True)
data.query('location >0',inplace=True)
data.query('check >0',inplace=True)
data.query('service >0',inplace=True)
data.query('business >0',inplace=True)
data.query('target != "neutro"',inplace=True)
data['id'] = np.arange(len(data))

cols=data.columns.tolist()
cols= cols[-1:]+cols[:-1]
data=data[cols]
data.to_csv("./csv/training_sentimental.csv",index=False)
all_data=len(data)
data.head(5)

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
0,0,hotel_231505,-1,-1,5,5,5,5,5,5,5,5,good
1,1,hotel_231505,-1,-1,1,1,1,3,2,1,3,3,good
2,2,hotel_231505,-1,-1,3,3,2,3,4,2,2,3,good
5,3,hotel_231505,4,4,4,4,4,4,4,4,3,5,good
7,4,hotel_231505,3,3,4,4,4,3,4,5,4,3,good


In [25]:
bad=data.query('target == "bad" and overall >0')
good=data.query('target == "good"')
bad_n=len(bad)
good_n=len(good)

print(float(good_n)/all_data,float(bad_n)/all_data)

(0.9428820463560844, 0.057117953643915545)


In [27]:
#numero di volte per ogni valore di clean dato good
clean_g_1=data.query('target == "good" and clean == 1')
clean_g_2=data.query('target == "good" and clean == 2')
clean_g_3=data.query('target == "good" and clean == 3')
clean_g_4=data.query('target == "good" and clean == 4')
clean_g_5=data.query('target == "good" and clean == 5')
clean_g=[len(clean_g_1),len(clean_g_2),len(clean_g_3),len(clean_g_4),len(clean_g_5)]
clean_g

#numero di volte per ogni valore di clean dato bad
clean_b_1=data.query('target == "bad" and clean == 1')
clean_b_2=data.query('target == "bad" and clean == 2')
clean_b_3=data.query('target == "bad" and clean == 3')
clean_b_4=data.query('target == "bad" and clean == 4')
clean_b_5=data.query('target == "bad" and clean == 5')
clean_b=[len(clean_b_1),len(clean_b_2),len(clean_b_3),len(clean_b_4),len(clean_b_5)]
clean_b

#numero di volte per ogni valore di value dato good
value_g_1=data.query('target == "good" and value == 1')
value_g_2=data.query('target == "good" and value == 2')
value_g_3=data.query('target == "good" and value == 3')
value_g_4=data.query('target == "good" and value == 4')
value_g_5=data.query('target == "good" and value == 5')
value_g=[len(value_g_1),len(value_g_2),len(value_g_3),len(value_g_4),len(value_g_5)]

#numero di volte per ogni valore di value dato bad
value_b_1=data.query('target == "bad" and value == 1')
value_b_2=data.query('target == "bad" and value == 2')
value_b_3=data.query('target == "bad" and value == 3')
value_b_4=data.query('target == "bad" and value == 4')
value_b_5=data.query('target == "bad" and value == 5')
value_b=[len(value_b_1),len(value_b_2),len(value_b_3),len(value_b_4),len(value_b_5)]

#numero di volte per ogni valore di room dato good
room_g_1=data.query('target == "good" and room == 1')
room_g_2=data.query('target == "good" and room == 2')
room_g_3=data.query('target == "good" and room == 3')
room_g_4=data.query('target == "good" and room == 4')
room_g_5=data.query('target == "good" and room == 5')
room_g=[len(room_g_1),len(room_g_2),len(room_g_3),len(room_g_4),len(room_g_5)]

#numero di volte per ogni valore di room dato bad
room_b_1=data.query('target == "bad" and room == 1')
room_b_2=data.query('target == "bad" and room == 2')
room_b_3=data.query('target == "bad" and room == 3')
room_b_4=data.query('target == "bad" and room == 4')
room_b_5=data.query('target == "bad" and room == 5')
room_b=[len(room_b_1),len(room_b_2),len(room_b_3),len(room_b_4),len(room_b_5)]

#numero di volte per ogni valore di location dato good
loc_g_1=data.query('target == "good" and location == 1')
loc_g_2=data.query('target == "good" and location == 2')
loc_g_3=data.query('target == "good" and location == 3')
loc_g_4=data.query('target == "good" and location == 4')
loc_g_5=data.query('target == "good" and location == 5')
loc_g=[len(loc_g_1),len(loc_g_2),len(loc_g_3),len(loc_g_4),len(loc_g_5)]

#numero di volte per ogni valore di location dato bad
loc_b_1=data.query('target == "bad" and location == 1')
loc_b_2=data.query('target == "bad" and location == 2')
loc_b_3=data.query('target == "bad" and location == 3')
loc_b_4=data.query('target == "bad" and location == 4')
loc_b_5=data.query('target == "bad" and location == 5')
loc_b=[len(loc_b_1),len(loc_b_2),len(loc_b_3),len(loc_b_4),len(loc_b_5)]

#numero di volte per ogni valore di check dato good
check_g_1=data.query('target == "good" and check == 1')
check_g_2=data.query('target == "good" and check == 2')
check_g_3=data.query('target == "good" and check == 3')
check_g_4=data.query('target == "good" and check == 4')
check_g_5=data.query('target == "good" and check == 5')
check_g=[len(check_g_1),len(check_g_2),len(check_g_3),len(check_g_4),len(check_g_5)]

#numero di volte per ogni valore di check dato bad
check_b_1=data.query('target == "bad" and check == 1')
check_b_2=data.query('target == "bad" and check == 2')
check_b_3=data.query('target == "bad" and check == 3')
check_b_4=data.query('target == "bad" and check == 4')
check_b_5=data.query('target == "bad" and check == 5')
check_b=[len(check_b_1),len(check_b_2),len(check_b_3),len(check_b_4),len(check_b_5)]

#numero di volte per ogni valore di service dato good
ser_g_1=data.query('target == "good" and service == 1')
ser_g_2=data.query('target == "good" and service == 2')
ser_g_3=data.query('target == "good" and service == 3')
ser_g_4=data.query('target == "good" and service == 4')
ser_g_5=data.query('target == "good" and service == 5')
ser_g=[len(ser_g_1),len(ser_g_2),len(ser_g_3),len(ser_g_4),len(ser_g_5)]

#numero di volte per ogni valore di service dato bad
ser_b_1=data.query('target == "bad" and service == 1')
ser_b_2=data.query('target == "bad" and service == 2')
ser_b_3=data.query('target == "bad" and service == 3')
ser_b_4=data.query('target == "bad" and service == 4')
ser_b_5=data.query('target == "bad" and service == 5')
ser_b=[len(ser_b_1),len(ser_b_2),len(ser_b_3),len(ser_b_4),len(ser_b_5)]

#numero di volte per ogni valore di business dato good
b_g_1=data.query('target == "good" and business == 1')
b_g_2=data.query('target == "good" and business == 2')
b_g_3=data.query('target == "good" and business == 3')
b_g_4=data.query('target == "good" and business == 4')
b_g_5=data.query('target == "good" and business == 5')
b_g=[len(b_g_1),len(b_g_2),len(b_g_3),len(b_g_4),len(b_g_5)]

#numero di volte per ogni valore di service dato bad
b_b_1=data.query('target == "bad" and business == 1')
b_b_2=data.query('target == "bad" and business == 2')
b_b_3=data.query('target == "bad" and business == 3')
b_b_4=data.query('target == "bad" and business == 4')
b_b_5=data.query('target == "bad" and business == 5')
b_b=[len(b_b_1),len(b_b_2),len(b_b_3),len(b_b_4),len(b_b_5)]

#create view to cpt
def generate_cpt(name,array_g,array_b):
    cpt= PrettyTable()
    cpt.field_names =["P("+name+"|target)",1,2,3,4,5]
    good_c=['good']
    bad_c=['bad']
    for i in range(len(array_g)):
        good_c.append(float(array_g[i])/good_n)
        bad_c.append(float(array_b[i])/bad_n)
    cpt.add_row(good_c)
    cpt.add_row(bad_c)
    return cpt

print(generate_cpt("clean",clean_g,clean_b))
print(generate_cpt("value",value_g,value_b))
print(generate_cpt("room",room_g,room_b))
print(generate_cpt("location",loc_g,loc_b))
print(generate_cpt("check",check_g,check_b))
print(generate_cpt("service",ser_g,ser_b))
print(generate_cpt("business",b_g,b_b))

+-----------------+-----------------+-----------------+-----------------+----------------+----------------+
| P(clean|target) |        1        |        2        |        3        |       4        |       5        |
+-----------------+-----------------+-----------------+-----------------+----------------+----------------+
|       good      | 0.0119231785081 | 0.0292231068181 | 0.0813304154247 | 0.233596196657 | 0.643927102592 |
|       bad       |  0.275615073186 |  0.201183431953 |  0.232326378075 | 0.161009031454 | 0.129866085332 |
+-----------------+-----------------+-----------------+-----------------+----------------+----------------+
+-----------------+-----------------+-----------------+----------------+----------------+-----------------+
| P(value|target) |        1        |        2        |       3        |       4        |        5        |
+-----------------+-----------------+-----------------+----------------+----------------+-----------------+
|       good      | 0.023431

In [29]:
#testing
data=pd.read_csv("./Testing/first_step_prepro/testing_initial_sentimental.csv",sep='[;,|]', engine='python')
data.query('overall > 0',inplace=True)
data.query('clean > 0',inplace=True)
data.query('value >0',inplace=True)
data.query('room >0',inplace=True)
data.query('location >0',inplace=True)
data.query('check >0',inplace=True)
data.query('service >0',inplace=True)
data.query('business >0',inplace=True)
data.query('target != "neutro"',inplace=True)
data['id'] = np.arange(len(data))

cols=data.columns.tolist()
cols= cols[-1:]+cols[:-1]
data=data[cols]
data.to_csv("./csv/testing_sentimental.csv",index=False)
all_data=len(data)
data.head(5)

,id,hotel,reader,helpful,overall,value,room,location,clean,check,service,business,target
2,0,hotel_80799,-1,-1,3,5,3,3,3,5,3,4,good
5,1,hotel_80799,1,1,1,1,1,1,1,3,2,1,good
6,2,hotel_80799,2,1,4,5,3,4,5,5,5,3,good
9,3,hotel_80799,-1,-1,1,1,1,1,3,2,3,2,bad
13,4,hotel_80799,3,3,1,2,1,3,2,5,5,1,bad


In [31]:
#inferenza con naive bayes con sentimental analysis
skel= GraphSkeleton()
nd =NodeData()

skel.load('./skeleton_sentimental.txt')
nd.load("./skeleton_sentimental.txt")

skel.toporder()
bn=DiscreteBayesianNetwork(skel, nd)

def inference(row):
    fn =TableCPDFactorization(bn)
    evidence =dict(clean=row[8],value=row[5],room=row[6],location=row[7],check=row[9],service=row[10],business=row[11])
    query= {'target' :['bad']}
    bad = fn.specificquery(query,evidence)
    good=1-bad
    return (good,bad)

list_result=[]
list_good_sent=[]
for i in range(len(data)):
    row=data.query('id=='+str(i))
    result_row=row.values.tolist()[0]
    good,bad=inference(result_row)
    list_good_sent.append(good)
    print(good,bad)
    if good < bad:
        list_result.append('bad')
    else:
        list_result.append('good')
data['target']=list_result
        

(0.20063685280457966, 0.7993631471954203)
(2.878688639684768e-07, 0.999999712131136)
(0.8426081442301987, 0.15739185576980128)
(7.274296986325979e-06, 0.9999927257030137)
(0.000588726831329045, 0.999411273168671)
(4.092661721744051e-07, 0.9999995907338278)
(8.090250555037315e-07, 0.9999991909749445)
(0.9831170008158527, 0.01688299918414722)
(0.2696250872070326, 0.7303749127929674)
(0.7483727310780395, 0.25162726892196047)
(0.3402490015419962, 0.6597509984580038)
(0.032610045490661155, 0.9673899545093388)
(0.0026018423942621993, 0.9973981576057378)
(0.0012196514757664145, 0.9987803485242336)
(0.006362325931454915, 0.9936376740685451)
(7.463610506297513e-07, 0.9999992536389494)
(1.5711279166552572e-07, 0.9999998428872083)
(1.8057073947197466e-08, 0.999999981942926)
(0.28889985987592615, 0.7111001401240739)
(1.8057073947197466e-08, 0.999999981942926)
(0.003959115483989639, 0.9960408845160104)
(5.664482972123075e-06, 0.9999943355170279)
(2.3272829652309923e-07, 0.9999997672717035)
(0.82545

(0.6913421205210657, 0.3086578794789343)
(0.9831170008158527, 0.01688299918414722)
(0.8941890928157619, 0.10581090718423819)
(0.8782938415667472, 0.12170615843325283)
(0.9751313638215968, 0.024868636178403212)
(0.8690795386501027, 0.13092046134989738)
(0.9831170008158527, 0.01688299918414722)
(0.9395306496893054, 0.06046935031069461)
(0.7610278651877262, 0.2389721348122738)
(0.3118131008277555, 0.6881868991722445)
(0.7560303160341505, 0.24396968396584948)
(0.7683444049702215, 0.23165559502977853)
(0.9831170008158527, 0.01688299918414722)
(0.8012638932207498, 0.19873610677925027)
(0.16717788343347884, 0.8328221165665212)
(2.556292170163843e-07, 0.999999744370783)
(0.6913421205210657, 0.3086578794789343)
(1.535987126599636e-06, 0.9999984640128734)
(0.00010985699711751895, 0.9998901430028825)
(0.9486131951558543, 0.05138680484414565)
(0.3006887296630658, 0.6993112703369342)
(1.8288184432613264e-05, 0.9999817118155674)
(0.9831170008158527, 0.01688299918414722)
(0.9654404109932881, 0.034559

(0.24656817913249862, 0.7534318208675014)
(0.77311982811371, 0.22688017188628998)
(1.03211451930596e-05, 0.9999896788548069)
(0.9569194630665493, 0.043080536933450674)
(0.19154507169743706, 0.8084549283025629)
(0.9572204957988649, 0.04277950420113511)
(0.912758213509464, 0.08724178649053607)
(0.01371709095403817, 0.9862829090459618)
(0.45334252835083955, 0.5466574716491605)
(1.5068126754114175e-06, 0.9999984931873246)
(0.8712808337984952, 0.12871916620150475)
(0.8978693144823908, 0.10213068551760923)
(0.4148181819781257, 0.5851818180218743)
(0.11278394434273109, 0.8872160556572689)
(2.8073147665397258e-05, 0.9999719268523346)
(0.011177150429795835, 0.9888228495702042)
(0.00014812757037119262, 0.9998518724296288)
(0.9831170008158527, 0.01688299918414722)
(0.061842865987090434, 0.9381571340129096)
(0.13025949865201836, 0.8697405013479816)
(0.10965617488422241, 0.8903438251157776)
(0.6815428419403518, 0.3184571580596483)
(0.6909830144572486, 0.30901698554275137)
(0.8978693144823908, 0.102

(0.010415852457390073, 0.9895841475426099)
(0.0002744206137718086, 0.9997255793862282)
(0.9486131951558543, 0.05138680484414565)
(0.974682751597515, 0.025317248402484963)
(0.9110178089918909, 0.08898219100810914)
(0.7454795922464887, 0.2545204077535113)
(0.8535196400856445, 0.14648035991435548)
(0.9831170008158527, 0.01688299918414722)
(3.2970912281493625e-06, 0.9999967029087719)
(0.0004829783152058287, 0.9995170216847942)
(1.07826722530735e-05, 0.9999892173277469)
(0.45379183725021177, 0.5462081627497882)
(0.9705767164024883, 0.029423283597511642)
(0.48501848637822464, 0.5149815136217754)
(0.4439447791291301, 0.5560552208708699)
(0.28889985987592615, 0.7111001401240739)
(0.8532067086836169, 0.14679329131638305)
(5.215130359781384e-07, 0.999999478486964)
(0.032333475233690345, 0.9676665247663097)
(0.06486383860393563, 0.9351361613960644)
(4.229882759609893e-06, 0.9999957701172404)
(0.012538316298944419, 0.9874616837010556)
(0.00014122309222697194, 0.999858776907773)
(0.0005362987224152

(0.9751313638215968, 0.024868636178403212)
(0.974682751597515, 0.025317248402484963)
(0.9831170008158527, 0.01688299918414722)
(0.850535525856274, 0.14946447414372596)
(0.9553325712942342, 0.04466742870576574)
(0.9831170008158527, 0.01688299918414722)
(0.9831170008158527, 0.01688299918414722)
(0.7168868964848526, 0.2831131035151474)
(0.9831170008158527, 0.01688299918414722)
(0.9362464005517165, 0.06375359944828356)
(0.9831170008158527, 0.01688299918414722)
(0.910147484912518, 0.08985251508748203)
(0.950717081613854, 0.049282918386145916)
(0.9831170008158527, 0.01688299918414722)
(0.9831170008158527, 0.01688299918414722)
(0.9751313638215968, 0.024868636178403212)
(0.9831170008158527, 0.01688299918414722)
(0.9285205669946539, 0.07147943300534614)
(0.9831170008158527, 0.01688299918414722)
(0.9751313638215968, 0.024868636178403212)
(0.9133398847579345, 0.08666011524206549)
(0.9831170008158527, 0.01688299918414722)
(0.34248407570301875, 0.6575159242969812)
(0.8887338661715103, 0.11126613382

...

(0.0038077611821443202, 0.9961922388178557)
(0.9485311257668713, 0.0514688742331287)
(0.1608351224135527, 0.8391648775864473)
(0.0001778597361508094, 0.9998221402638492)
(0.9751313638215968, 0.024868636178403212)
(4.7796208526573736e-06, 0.9999952203791473)
(0.9362464005517165, 0.06375359944828356)
(0.40692162303830637, 0.5930783769616936)
(0.10570901107114228, 0.8942909889288577)
(1.8057073947197466e-08, 0.999999981942926)
(0.050654674976733705, 0.9493453250232663)
(0.29779186964988325, 0.7022081303501168)
(0.9553325712942342, 0.04466742870576574)
(0.00019976898032847412, 0.9998002310196715)
(0.28889985987592615, 0.7111001401240739)
(0.07099016524744994, 0.9290098347525501)
(0.007207831054754643, 0.9927921689452454)
(0.019096809658007086, 0.9809031903419929)
(0.04149130150193636, 0.9585086984980636)
(0.01757750764553434, 0.9824224923544657)
(0.9129865808360127, 0.08701341916398733)
(2.1669278324476515e-05, 0.9999783307216755)
(0.9133398847579345, 0.08666011524206549)
(0.00994335237725

(0.912758213509464, 0.08724178649053607)
(0.9831170008158527, 0.01688299918414722)
(0.026988235450942977, 0.973011764549057)
(0.001590119725132233, 0.9984098802748678)
(0.5088517783424538, 0.4911482216575463)
(0.046279596476589524, 0.9537204035234105)
(0.17830712617891287, 0.8216928738210871)
(1.8057073947197466e-08, 0.999999981942926)
(0.531608079062572, 0.468391920937428)
(0.6494948592245094, 0.35050514077549055)
(0.8966993576468401, 0.10330064235315996)
(0.22642980941783553, 0.7735701905821645)
(0.5592415441040524, 0.4407584558959477)
(0.9399455642053496, 0.06005443579465037)
(0.0012112292501782473, 0.9987887707498218)
(3.1987988359793995e-05, 0.9999680120116402)
(0.6682189103435838, 0.3317810896564162)
(0.8961753469055529, 0.10382465309444715)
(0.6433665590887683, 0.3566334409112317)
(0.8254589000846926, 0.17454109991530742)
(0.0006888712044931244, 0.9993111287955069)
(0.9591851713698545, 0.04081482863014551)
(0.01722129244348669, 0.9827787075565133)
(0.09463869764255628, 0.9053613

(2.2098514120827417e-05, 0.9999779014858792)
(1.3100086560302593e-05, 0.9999868999134397)
(0.9643843383202898, 0.03561566167971015)
(0.9700148703404271, 0.029985129659572892)
(0.006146883021983407, 0.9938531169780166)
(0.6815056295593294, 0.31849437044067064)
(0.22942344974170492, 0.7705765502582951)
(0.8337971590835236, 0.16620284091647647)
(0.9242696482688427, 0.07573035173115727)
(0.741539134320397, 0.2584608656796031)
(0.13419010475868942, 0.8658098952413106)
(0.012019049632618128, 0.9879809503673819)
(0.8978693144823908, 0.10213068551760923)
(5.9167723272191e-05, 0.9999408322767278)
(0.0007191700481870011, 0.999280829951813)
(0.026988235450942977, 0.973011764549057)
(3.7893786197784607e-07, 0.999999621062138)
(0.9628584421982127, 0.03714155780178727)
(0.89488031236968, 0.10511968763032)
(0.4275747604967648, 0.5724252395032352)
(0.9831170008158527, 0.01688299918414722)
(0.9575631827509956, 0.04243681724900442)
(0.9237552145078621, 0.07624478549213791)
(0.9174868183575537, 0.0825131

(0.9480067745974612, 0.0519932254025388)
(0.8811318785848867, 0.11886812141511324)
(0.7683444049702215, 0.23165559502977853)
(0.06387724905719683, 0.9361227509428032)
(0.3344762380449916, 0.6655237619550084)
(0.8811318785848867, 0.11886812141511324)
(0.8065944456725702, 0.19340555432742976)
(0.0006719177004885779, 0.9993280822995114)
(0.05108722160565882, 0.9489127783943412)
(0.007484638481589201, 0.9925153615184108)
(0.08075906090085216, 0.9192409390991478)
(0.9362464005517165, 0.06375359944828356)
(0.21356945681728956, 0.7864305431827104)
(0.0002453355470256602, 0.9997546644529743)
(0.9474388018916251, 0.05256119810837492)
(0.8803623416985964, 0.11963765830140363)
(0.003886034323723697, 0.9961139656762763)
(0.9751313638215968, 0.024868636178403212)
(0.9831170008158527, 0.01688299918414722)
(0.8915206354179812, 0.10847936458201875)
(0.8915206354179812, 0.10847936458201875)
(0.5592415441040524, 0.4407584558959477)
(0.07494186093887423, 0.9250581390611258)
(0.4896016730234818, 0.5103983

(0.8864871421235941, 0.11351285787640586)
(0.8985392205469184, 0.10146077945308163)
(0.8171846519500792, 0.1828153480499208)
(0.556413877871597, 0.443586122128403)
(0.8965929712353399, 0.10340702876466011)
(0.7023595393961717, 0.29764046060382826)
(0.00029006265031739087, 0.9997099373496826)
(0.0030696294856064066, 0.9969303705143936)
(0.8985392205469184, 0.10146077945308163)
(0.10600829489603014, 0.8939917051039699)
(4.578909935659681e-07, 0.9999995421090064)
(0.974682751597515, 0.025317248402484963)
(0.00010912773281879229, 0.9998908722671812)
(0.00027980604204336235, 0.9997201939579566)
(8.666173803273391e-05, 0.9999133382619673)
(0.3418788343090132, 0.6581211656909868)
(0.003202749934358984, 0.996797250065641)
(0.9831170008158527, 0.01688299918414722)
(0.9628584421982127, 0.03714155780178727)
(0.8514645460873611, 0.14853545391263895)
(0.9319629008148825, 0.0680370991851175)
(0.4176528208914533, 0.5823471791085467)
(0.010472135048054998, 0.989527864951945)
(0.8514645460873611, 0.148

In [33]:
data['prob']=list_good_sent
data.to_csv("./csv/result_sentimental/testing_sentimental_naivebayes.csv",index=False)